We want to calculate the covariance matrix of our background data.



In [1]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [27]:
full_time_series = torch.from_numpy(np.loadtxt("./data/background_0.dat", dtype=np.float32))
full_time_series = full_time_series.to(device)

In [38]:
time_series = full_time_series[0:100000]

In [53]:
#time_series = torch.normal(10, 1, size=(1, 1000000),dtype=torch.float32)[0]

# sliding window of size 5, rows are the waveform slices
windows = time_series.unfold(0,5,1)

# windows.T is rows of our random variables, take mean per row
means = torch.mean(windows.T, axis=1)

# windows - means subtracts means of each var from each window, so now it is window of differences from mean
cov_mat = (windows - means).T @ (windows - means) * (1.0 / (windows.shape[0]))
#inv_cov_mat = torch.inverse(cov_mat)

print(cov_mat)
#print(inv_cov_mat)

#print(cov_mat @ inv_cov_mat)

tensor([[21.2198,  0.1462,  0.3845,  0.1484,  0.2283],
        [ 0.1462, 21.2198,  0.1462,  0.3844,  0.1483],
        [ 0.3845,  0.1462, 21.2197,  0.1459,  0.3842],
        [ 0.1484,  0.3844,  0.1459, 21.2191,  0.1456],
        [ 0.2283,  0.1483,  0.3842,  0.1456, 21.2192]])


In [52]:
# This algorithm is numerically unstable



sliding_window_size = 5

# sliding window of size 5, rows are the waveform slices
windows = time_series.unfold(0, sliding_window_size, 1)

# windows.T is rows of our random variables, take mean per row
means = torch.mean(windows.T, axis=1)

cov_row_one = F.conv1d(time_series.view(1, 1, -1), time_series.view(1, 1, -1)[:,:,0:time_series.shape[0] + 1 - sliding_window_size]) / (time_series.shape[0] + 1 - sliding_window_size) - means * means[0]

print(cov_row_one[0,0])

tensor([27.7363,  3.1055,  3.3945,  2.8398,  2.9883])
